<a href="https://colab.research.google.com/github/Vikas-KM/tensorflow-learning/blob/master/cnn-cifar20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10

os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

In [4]:
# load the dataset
(X_train, y_train),(X_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 2s 0us/step


In [11]:
# convert to float32 and normalize the data
X_train = X_train.astype('float32')/255.0 
X_test = X_test.astype('float32')/255.0

In [20]:
# Sequential
model = keras.Sequential(
    [
     keras.Input(shape=(32,32,3)),
     layers.Conv2D(32, 3, padding='valid', activation='relu'),
     layers.MaxPooling2D(pool_size=(2,2)),
     layers.Conv2D(64,3, activation='relu'),
     layers.MaxPooling2D(),
     layers.Conv2D(128,3, activation='relu'),
     layers.Flatten(),
     layers.Dense(64, activation='relu'),
     layers.Dense(10),
    ]
)

In [21]:
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 4, 4, 128)         73856     
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)               

In [22]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(learning_rate=3e-4),
    metrics = ['accuracy']
)

In [23]:
model.fit(X_train, y_train, batch_size=64, epochs=10, verbose=2)

Epoch 1/10
782/782 - 60s - loss: 1.7090 - accuracy: 0.3774
Epoch 2/10
782/782 - 59s - loss: 1.3590 - accuracy: 0.5145
Epoch 3/10
782/782 - 59s - loss: 1.2483 - accuracy: 0.5606
Epoch 4/10
782/782 - 59s - loss: 1.1516 - accuracy: 0.5997
Epoch 5/10
782/782 - 59s - loss: 1.0818 - accuracy: 0.6227
Epoch 6/10
782/782 - 59s - loss: 1.0193 - accuracy: 0.6470
Epoch 7/10
782/782 - 58s - loss: 0.9681 - accuracy: 0.6642
Epoch 8/10
782/782 - 58s - loss: 0.9188 - accuracy: 0.6823
Epoch 9/10
782/782 - 59s - loss: 0.8788 - accuracy: 0.6966
Epoch 10/10
782/782 - 58s - loss: 0.8423 - accuracy: 0.7086


In [24]:
model.evaluate(X_test, y_test, batch_size=64, verbose=2)

157/157 - 3s - loss: 0.9447 - accuracy: 0.6748


[0.9447250962257385, 0.6747999787330627]

In [32]:
# Functional
def my_model():
  inputs = keras.Input(shape=(32,32,3))
  x = layers.Conv2D(32,3)(inputs)
  x = layers.BatchNormalization()(x)
  x = keras.activations.relu(x)
  x = layers.MaxPooling2D()(x)
  x = layers.Conv2D(64,5, padding='same')(x)
  x = layers.BatchNormalization()(x)
  x = keras.activations.relu(x)
  x = layers.Conv2D(128,3)(x)
  x = layers.BatchNormalization()(x)
  x = keras.activations.relu(x)
  x = layers.Flatten()(x)
  x = layers.Dense(64, activation='relu')(x)
  outputs = layers.Dense(10)(x)
  model = keras.Model(inputs=inputs, outputs=outputs)
  return model


In [33]:
model = my_model()

In [37]:
print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
batch_normalization_5 (Batch (None, 30, 30, 32)        128       
_________________________________________________________________
tf.nn.relu_5 (TFOpLambda)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 15, 15, 64)        51264     
_________________________________________________________________
batch_normalization_6 (Batch (None, 15, 15, 64)        256 

In [34]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(learning_rate=3e-4),
    metrics=['accuracy'],
)

In [35]:
model.fit(X_train, y_train, batch_size=64, epochs=10, verbose=2)

Epoch 1/10
782/782 - 244s - loss: 1.3543 - accuracy: 0.5156
Epoch 2/10
782/782 - 243s - loss: 0.9270 - accuracy: 0.6710
Epoch 3/10
782/782 - 245s - loss: 0.7677 - accuracy: 0.7298
Epoch 4/10
782/782 - 241s - loss: 0.6535 - accuracy: 0.7670
Epoch 5/10
782/782 - 241s - loss: 0.5757 - accuracy: 0.7984
Epoch 6/10
782/782 - 244s - loss: 0.5021 - accuracy: 0.8236
Epoch 7/10
782/782 - 240s - loss: 0.4311 - accuracy: 0.8497
Epoch 8/10
782/782 - 242s - loss: 0.3701 - accuracy: 0.8708
Epoch 9/10
782/782 - 239s - loss: 0.3166 - accuracy: 0.8917
Epoch 10/10
782/782 - 239s - loss: 0.2637 - accuracy: 0.9095


In [36]:
model.evaluate(X_test, y_test, batch_size=64, verbose=2)

157/157 - 12s - loss: 1.0833 - accuracy: 0.6864


[1.0833382606506348, 0.6863999962806702]